In [40]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [41]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [42]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)

In [43]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()
loansContract.functions.setAccountsContractAddress(accountsContractAdd).transact()

HexBytes('0xd7625c362aba54e08ba50e313d465a75b9aada70e8027276e58e3f3101db98a0')

HexBytes('0x8db49e4fc468bc971944a65b911eb602da9f3def4439ead18ee44ac6db2af2aa')

HexBytes('0x89afed1ed9127ccc15fcb3a9497546b1a681d4d80973f5df8580fc793ead4426')

In [44]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0x8732324668a289749fef79160e6798a99475467021b3100b963f339d8268a147')

HexBytes('0x4950a84ecff2f1703e8fa8fe5ecc900dad8ec2474638ba989cab937e8dbae6c5')

HexBytes('0x3594e754e7f4d87b697f6ec3ef36773254f673d350e49867982851e0187ddbb9')

HexBytes('0x8b542a5803ae4a702126c0123ed3c2ae4650be78de068ed1f2ea7e40befeb4d7')

In [45]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract, _installmentsNum ,  _interest):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount, _installmentsNum ,  _interest
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [46]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [47]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
loaner = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

'0xDED00A7f19D445B2c6D3ddFffbf78BB490771937'

In [48]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

'0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290'

## Get pending loans

In [49]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [19]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


['124192 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590928764 10 20 ',
 '659367 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590928769 8 17 ',
 '866314 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590928773 9 22 ',
 '601249 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590928775 14 10 ']

## Confirm or reject loans

In [55]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 1200000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 354595
c/r?c
for id: 333321
c/r?r
for id: 972988
c/r?c
for id: 16182
c/r?r


# Create a new loan

In [53]:
loaner
# Create a new loan
if loanerIndex != -1:
    loanerType = accountsConract.functions.getType(loanerIndex).call()
    if loanerType:
        # We will change this to inputs later
        amount=random.randint(1000,1000000)
        if amount%2!=0:
            amount+=1
        installmentsNum=random.randint(6,15)
        interest=random.randint(5,35)
        amount+=(amount*interest)//100
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=amount, _organizationContract=organizationContract, _installmentsNum=installmentsNum, _interest=interest):
            print("Loan created")
else:
    print("This account is not registered in our system.")

'0xDED00A7f19D445B2c6D3ddFffbf78BB490771937'

Enter the password of the organization: ········
Loan created


## Get user loans

In [56]:
def getLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions. getMyLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [57]:
def getLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [58]:
getLoansList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['354595 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590932815 6 21 ',
 '972988 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590932820 12 12 ']

In [59]:
def getInstallments(_loanContract, _loanieAddress, _privateKey):
    myId = 1590932820
    transaction = _loanContract.functions.getMyInstallments(myId
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _loanContract.events.getLoanInstallments().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [60]:
def getinstallmentsList(_accountIndex, _loanContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_loanContract, _loanieAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [61]:
getinstallmentsList(loanieIndex,loansContract,accountsConract,loanie)

Enter your password: ········


['81082 1593524848 0 False ',
 '81082 1596116848 0 False ',
 '81082 1598708848 0 False ',
 '81082 1601300848 0 False ',
 '81082 1603892848 0 False ',
 '81082 1606484848 0 False ',
 '81082 1609076848 0 False ',
 '81082 1611668848 0 False ',
 '81082 1614260848 0 False ',
 '81082 1616852848 0 False ',
 '81082 1619444848 0 False ',
 '81086 1622036848 0 False ']

## Get Organization Loans

In [62]:
def getLoanerLoans(_organizationContract, _loanerAddress, _privateKey):
    transaction = _organizationContract.functions. getLoans( 
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanerAddress,
    'nonce': web3.eth.getTransactionCount(_loanerAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _organizationContract.events.getLoanerLoans().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [63]:
def getLoanerLoansList(_accountIndex,_organizationContract,_accountsContract, _loanerAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoanerLoans(_organizationContract, _loanerAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [64]:
getLoanerLoansList(loanerIndex,organizationContract,accountsConract,loaner)

Enter your password: ········


['354595 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590932815 6 21 ',
 '972988 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1590932820 12 12 ']

## Get installment for the orgianization

In [65]:
def getLoanerInstallmentsList(_accountIndex, _loanContract, _accountsContract, _loanerAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_loanContract, _loanerAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [66]:
getLoanerInstallmentsList(loanerIndex,loansContract,accountsConract,loaner)

Enter your password: ········


['81082 1593524848 0 False ',
 '81082 1596116848 0 False ',
 '81082 1598708848 0 False ',
 '81082 1601300848 0 False ',
 '81082 1603892848 0 False ',
 '81082 1606484848 0 False ',
 '81082 1609076848 0 False ',
 '81082 1611668848 0 False ',
 '81082 1614260848 0 False ',
 '81082 1616852848 0 False ',
 '81082 1619444848 0 False ',
 '81086 1622036848 0 False ']

## Confirm paid installment

In [67]:
def confirmInstallment (_loanerIndex,_loansContract,_organizationContract,_loaner,_accountsConract):
    
    installments = getLoanerInstallmentsList(_loanerIndex,_loansContract,_accountsConract,_loaner)
    print(installments)
    choice = input("enter the index of installment")
    if int(choice) >= len (installments):
        print("wrong index")
        return
    myid = 1590932820
    transaction = _organizationContract.functions.confirmInstallment(int(choice),myid
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    })    
    privateKey = getpass("Enter your password: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    return
    

In [68]:
confirmInstallment(loanerIndex,loansContract,organizationContract,loaner,accountsConract)

Enter your password: ········
['81082 1593524848 0 False ', '81082 1596116848 0 False ', '81082 1598708848 0 False ', '81082 1601300848 0 False ', '81082 1603892848 0 False ', '81082 1606484848 0 False ', '81082 1609076848 0 False ', '81082 1611668848 0 False ', '81082 1614260848 0 False ', '81082 1616852848 0 False ', '81082 1619444848 0 False ', '81086 1622036848 0 False ']
enter the index of installment0
Enter your password: ········
